In [68]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import json 
from pandas.io.json import json_normalize
from pyspark.sql import SparkSession

## Extract I94 Immigration Data:

In [ ]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [ ]:
#write to parquet
df_spark.write.parquet("source_data/sas_data")

In [ ]:
df_spark=spark.read.parquet("sas_data")

### count total column

In [ ]:
df_spark.count()

### view column types

In [ ]:
df_spark.printSchema()

### view sample table

In [ ]:
df_spark.show(2)

## Extract I94 Immigration Data __SAMPLE_DATA__

In [42]:
immigration_df = pd.read_csv('source_data/immigration_data_sample.csv') 

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [123]:
immigration_df.dtypes

Unnamed: 0      int64
cicid         float64
i94yr         float64
i94mon        float64
i94cit        float64
i94res        float64
i94port        object
arrdate       float64
i94mode       float64
i94addr        object
depdate       float64
i94bir        float64
i94visa       float64
count         float64
dtadfile        int64
visapost       object
occup          object
entdepa        object
entdepd        object
entdepu       float64
matflag        object
biryear       float64
dtaddto        object
gender         object
insnum        float64
airline        object
admnum        float64
fltno          object
visatype       object
dtype: object

In [124]:
immigration_df.count()

Unnamed: 0    1000
cicid         1000
i94yr         1000
i94mon        1000
i94cit        1000
i94res        1000
i94port       1000
arrdate       1000
i94mode       1000
i94addr        941
depdate        951
i94bir        1000
i94visa       1000
count         1000
dtadfile      1000
visapost       382
occup            4
entdepa       1000
entdepd        954
entdepu          0
matflag        954
biryear       1000
dtaddto       1000
gender         859
insnum          35
airline        967
admnum        1000
fltno          992
visatype      1000
dtype: int64

In [125]:

sample_data_df = immigration_df.head(3)

In [126]:
sample_data_df.T

,0,1,2
Unnamed: 0,2027561,2171295,589494
cicid,4.08432e+06,4.42264e+06,1.1956e+06
i94yr,2016,2016,2016
i94mon,4,4,4
i94cit,209,582,148
i94res,209,582,112
i94port,HHW,MCA,OGG
arrdate,20566,20567,20551
i94mode,1,1,1
i94addr,HI,TX,FL


In [127]:
new_columns = [
'unnamed',
'cic_id',
'year',
'month',
'i94_city',
'i94_resident',
'i94_airport',
'arr_date',
'arrival_mode',
'i94_address',
'departure_date',
'age',
'visa_type',
'count',
'dtadfile',
'visa_codes',
'occupaption',
'arrival_flag',
'departure_flag',
'update_flag',
'match_flag',
'birthyear',
'dtaddto',
'gender',
'ins_number',
'airline',
'admission_nbr',
'flight_nbr',
'visa_type'
]
sample_data_df  = sample_data_df.rename(columns=dict(zip(sample_data_df.columns, new_columns)))
sample_data_df

,unnamed,cic_id,year,month,i94_city,i94_resident,i94_airport,arr_date,arrival_mode,i94_address,...,update_flag,match_flag,birthyear,dtaddto,gender,ins_number,airline,admission_nbr,flight_nbr,visa_type
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT


In [128]:
sample_data_df.head().T

,0,1,2
unnamed,2027561,2171295,589494
cic_id,4.08432e+06,4.42264e+06,1.1956e+06
year,2016,2016,2016
month,4,4,4
i94_city,209,582,148
i94_resident,209,582,112
i94_airport,HHW,MCA,OGG
arr_date,20566,20567,20551
arrival_mode,1,1,1
i94_address,HI,TX,FL


In [145]:
sample_data_df['arrival_date'] = (pd.to_timedelta(sample_data_df["arr_date"],unit="D") + pd.Timestamp('1960-1-1')).dt.date


In [146]:
sample_data_df.head().T


,0,1,2
unnamed,2027561,2171295,589494
cic_id,4.08432e+06,4.42264e+06,1.1956e+06
year,2016,2016,2016
month,4,4,4
i94_city,209,582,148
i94_resident,209,582,112
i94_airport,HHW,MCA,OGG
arr_date,20566,20567,20551
arrival_mode,1,1,1
i94_address,HI,TX,FL


## Extract World Temperature Data:

## Use Dask for data > 500 MB

In [5]:
from dask.distributed import Client, progress
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB')
client

Client Scheduler: tcp://127.0.0.1:60490,Cluster Workers: 2 Cores: 4 Memory: 2.00 GB


In [89]:
temperature_df = dd.read_csv('source_data/climate-change-earth-surface-temperature-data/GlobalLandTemperaturesByCity.csv') 

### count total column

In [90]:
temperature_df.count().compute()

dt                               8599212
AverageTemperature               8235082
AverageTemperatureUncertainty    8235082
City                             8599212
Country                          8599212
Latitude                         8599212
Longitude                        8599212
dtype: int64

### view column types

In [91]:
temperature_df.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                              object
Country                           object
Latitude                          object
Longitude                         object
dtype: object

### view sample table

In [92]:
US_temperature_df = temperature_df[temperature_df['Country']=='United States'].compute()
US_temperature_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
47555,1820-01-01,2.101,3.217,Abilene,United States,32.95N,100.53W
47556,1820-02-01,6.926,2.853,Abilene,United States,32.95N,100.53W
47557,1820-03-01,10.767,2.395,Abilene,United States,32.95N,100.53W
47558,1820-04-01,17.989,2.202,Abilene,United States,32.95N,100.53W
47559,1820-05-01,21.809,2.036,Abilene,United States,32.95N,100.53W


## Extract U.S. City Demographic Data

## Use simple dataFrame

In [187]:
cities_df = pd.read_json('source_data/us-cities-demographics.json') 
cities_df = json_normalize(cities_df['fields'])
cities_df.head()

,average_household_size,city,count,female_population,foreign_born,male_population,median_age,number_of_veterans,race,state,state_code,total_population
0,2.73,Newark,76402,143873.0,86253.0,138040.0,34.6,5829.0,White,New Jersey,NJ,281913
1,2.40,Peoria,1343,62432.0,7517.0,56229.0,33.1,6634.0,American Indian and Alaska Native,Illinois,IL,118661
2,2.77,O'Fallon,2583,43270.0,3269.0,41762.0,36.0,5783.0,Hispanic or Latino,Missouri,MO,85032
3,2.48,Hampton,70303,70240.0,6204.0,66214.0,35.5,19638.0,Black or African-American,Virginia,VA,136454
4,2.29,Lakewood,33630,76576.0,14169.0,76013.0,37.7,9988.0,Hispanic or Latino,Colorado,CO,152589


### count total column

In [188]:
cities_df.count()

average_household_size    2875
city                      2891
count                     2891
female_population         2888
foreign_born              2878
male_population           2888
median_age                2891
number_of_veterans        2878
race                      2891
state                     2891
state_code                2891
total_population          2891
dtype: int64

### view column types

In [189]:
cities_df.dtypes

average_household_size    float64
city                       object
count                       int64
female_population         float64
foreign_born              float64
male_population           float64
median_age                float64
number_of_veterans        float64
race                       object
state                      object
state_code                 object
total_population            int64
dtype: object

### view sample table

## Extract Airport Code Table:

In [70]:
airport_df = pd.read_csv('source_data/airport-codes_csv.csv') 
airport_df = airport_df[airport_df['iso_country']=='US'].head()


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [77]:
airport_df[airport_df['iata_code'].notnull()].head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates


### count total column

In [39]:
airport_df.count()

ident           55819
type            55819
name            55819
elevation_ft    48663
continent       27853
iso_country     55573
iso_region      55819
municipality    50064
gps_code        41092
iata_code        9230
local_code      29032
coordinates     55819
dtype: int64

### view column types

In [40]:
airport_df.dtypes

ident            object
type             object
name             object
elevation_ft    float64
continent        object
iso_country      object
iso_region       object
municipality     object
gps_code         object
iata_code        object
local_code       object
coordinates      object
dtype: object

### view sample table